In [112]:
# 셀 1: 라이브러리 및 경로 설정

import pandas as pd
import numpy as np

# 🔹 파일 경로 설정 (필요하면 네 환경에 맞게 수정)
COHORT_PATH = "/content/drive/MyDrive/Colab Notebooks/cohort_ver48_remove_trash_col.csv"
EDSTAYS_PATH = "/content/drive/MyDrive/Colab Notebooks/ed_csvs/edstays.csv"
OUTPUT_PATH = "/content/drive/MyDrive/Colab Notebooks/cohort_ver180_static_vars.csv"


In [113]:
# 셀 2: 데이터 로딩

df = pd.read_csv(COHORT_PATH)
ed = pd.read_csv(EDSTAYS_PATH)

print("[INFO] cohort_ver48 shape:", df.shape)
print("[INFO] edstays shape:", ed.shape)

# 상위 3행 정도만 확인
display(df.head(3))
display(ed.head(3))


[INFO] cohort_ver48 shape: (1930, 36)
[INFO] edstays shape: (425087, 9)


,subject_id,hadm_id,icu_stay_id,icu_intime,icu_outtime,gender,race,arrival_transport,disposition,admittime,...,pci_starttime,pci_endtime,first_antithrombotic_charttime,dischtime,age,cci_score,cci_category,stay_id,hfrs_score,hfrs_category
0,10010058,26359957,39919549,2147-11-18 0:50,2147-11-18 3:19,M,ASIAN - SOUTH EAST ASIAN,AMBULANCE,ADMITTED,2147-11-18 2:34,...,2147-11-18 5:34,2147-11-19 8:53,2147-11-18 5:00,2147-11-19 4:00,80,1,1_2,39919549,7.1,5_15
1,10012438,22764825,35750515,2178-06-07 19:33,2178-06-07 21:57,M,WHITE,AMBULANCE,ADMITTED,2179-07-12 5:56,...,2178-06-07 21:45,2178-06-08 15:51,2178-06-07 23:00,2178-06-10 15:30,65,1,1_2,35750515,3.8,lt5
2,10013310,21243435,31736720,2153-05-26 8:56,2153-05-26 14:18,F,BLACK/AFRICAN,UNKNOWN,ADMITTED,2153-06-10 11:55,...,NaN,NaN,2153-05-26 19:00,2153-06-05 19:30,70,1,1_2,31736720,4.5,lt5


,subject_id,hadm_id,stay_id,intime,outtime,gender,race,arrival_transport,disposition
0,10000032,22595853.0,33258284,2180-05-06 19:17:00,2180-05-06 23:30:00,F,WHITE,AMBULANCE,ADMITTED
1,10000032,22841357.0,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,F,WHITE,AMBULANCE,ADMITTED
2,10000032,25742920.0,35968195,2180-08-05 20:58:00,2180-08-06 01:44:00,F,WHITE,AMBULANCE,ADMITTED


In [114]:
# === 3. ED arrival_transport 조인 (첫 방문 기준) ===

import pandas as pd

# 2번 셀까지에서 이미 cohort 기반 df가 있다고 가정 (예: df = cohort.copy())

# 3-1) edstays 불러오기
ed_path = "/content/drive/MyDrive/Colab Notebooks/ed_csvs/edstays.csv"
ed = pd.read_csv(ed_path)

# 3-2) 시간 컬럼을 datetime으로 변환
ed["intime"] = pd.to_datetime(ed["intime"], errors="coerce")

# 3-3) subject_id, hadm_id, intime 기준으로 정렬 후,
#      각 (subject_id, hadm_id) 조합에서 첫 ED 방문 1개만 남기기
ed_first = (
    ed.sort_values(["subject_id", "hadm_id", "intime"])
      .groupby(["subject_id", "hadm_id"], as_index=False)
      .first()[["subject_id", "hadm_id", "arrival_transport"]]
)

print("[INFO] edstays 원본 shape:", ed.shape)
print("[INFO] edstays 첫 방문만 남긴 shape:", ed_first.shape)

# 🔹 3-3.5) 기존 df 안에 있던 arrival_transport 관련 컬럼은 모두 제거
to_drop = [c for c in df.columns if c.startswith("arrival_transport")]
if to_drop:
    print("[INFO] 기존 arrival_transport 관련 컬럼 삭제:", to_drop)
    df = df.drop(columns=to_drop)

# 3-4) cohort(df)와 조인 (이제는 행이 절대 늘어나지 않음 + 컬럼 이름 충돌 없음)
df = df.merge(ed_first, on=["subject_id", "hadm_id"], how="left")

print("[INFO] 조인 이후 df shape:", df.shape)
print(df[["subject_id", "hadm_id", "arrival_transport"]].head())


[INFO] edstays 원본 shape: (425087, 9)
[INFO] edstays 첫 방문만 남긴 shape: (202441, 3)
[INFO] 기존 arrival_transport 관련 컬럼 삭제: ['arrival_transport']
[INFO] 조인 이후 df shape: (1930, 36)
   subject_id   hadm_id arrival_transport
0    10010058  26359957         AMBULANCE
1    10012438  22764825         AMBULANCE
2    10013310  21243435           UNKNOWN
3    10013310  27682188         AMBULANCE
4    10033552  26487381         AMBULANCE


In [115]:
# 셀 4: 시간 관련 컬럼 datetime 변환

time_cols = [
    "ed_intime",
    "first_ecg_charttime",
    "first_troponin_charttime",
    "first_antithrombotic_charttime",
    "pci_starttime",
]

for col in time_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")
        print(f"[INFO] {col} → datetime 변환 완료")
    else:
        print(f"[WARN] 시간 컬럼 {col} 이(가) df에 없습니다.")

df[time_cols].head(3)


[INFO] ed_intime → datetime 변환 완료
[INFO] first_ecg_charttime → datetime 변환 완료
[INFO] first_troponin_charttime → datetime 변환 완료
[INFO] first_antithrombotic_charttime → datetime 변환 완료
[INFO] pci_starttime → datetime 변환 완료


,ed_intime,first_ecg_charttime,first_troponin_charttime,first_antithrombotic_charttime,pci_starttime
0,2147-11-18 00:50:00,2119-04-03 09:52:00,2147-11-18 05:43:00,2147-11-18 05:00:00,2147-11-18 05:34:00
1,2178-06-07 19:33:00,2167-11-11 13:58:00,NaT,2178-06-07 23:00:00,2178-06-07 21:45:00
2,2153-05-26 08:56:00,2118-05-26 08:28:00,2153-05-26 14:45:00,2153-05-26 19:00:00,NaT


In [116]:
# 셀 5: gender 인코딩 (0: Female, 1: Male)

def encode_gender(x):
    if pd.isna(x):
        return -1   # 결측치 → -1
    s = str(x).strip().upper()
    if s.startswith("F"):
        return 0
    if s.startswith("M"):
        return 1
    return -1       # unknown -> -1

if "gender" not in df.columns:
    raise ValueError("df에 'gender' 컬럼이 없습니다.")

df["gender_enc"] = df["gender"].apply(encode_gender).astype(int)

print("[INFO] gender 인코딩 분포:")
print(df["gender_enc"].value_counts(dropna=False))
df[["gender", "gender_enc"]].head(5)


[INFO] gender 인코딩 분포:
gender_enc
1    1239
0     691
Name: count, dtype: int64


,gender,gender_enc
0,M,1
1,M,1
2,F,0
3,F,0
4,F,0


In [117]:
# 셀 6 (수정): arrival_transport 인코딩
# 0: UNKNOWN/OTHER
# 1: WALK IN
# 2: AMBULANCE
# 3: HELICOPTER

import numpy as np

if "arrival_transport" not in df.columns:
    raise ValueError("df에 'arrival_transport' 컬럼이 없습니다. 조인/rename 과정을 다시 확인하세요.")

# 문자열로 변환 + 대문자
s = df["arrival_transport"].astype("string").str.upper()

# 패턴별 마스크
mask_amb  = s.str.contains("AMBULANCE", na=False) | s.str.contains("AMB", na=False)
mask_heli = s.str.contains("HELICOPTER", na=False) | s.str.contains("HELI", na=False)
mask_walk = s.str.contains("WALK", na=False)       # WALK IN, WALK-IN 등 포함

# np.select로 인코딩
df["arrival_transport_enc"] = np.select(
    [mask_amb, mask_heli, mask_walk],
    [2,        3,         1],
    default=0  # 나머지는 UNKNOWN/OTHER → 0
).astype(int)

print("[INFO] arrival_transport 인코딩 분포:")
print(df["arrival_transport_enc"].value_counts(dropna=False))
df[["arrival_transport", "arrival_transport_enc"]].head(10)


[INFO] arrival_transport 인코딩 분포:
arrival_transport_enc
2    1046
1     494
0     372
3      18
Name: count, dtype: int64


,arrival_transport,arrival_transport_enc
0,AMBULANCE,2
1,AMBULANCE,2
2,UNKNOWN,0
3,AMBULANCE,2
4,AMBULANCE,2
5,UNKNOWN,0
6,AMBULANCE,2
7,AMBULANCE,2
8,AMBULANCE,2
9,WALK IN,1


In [118]:
# 셀 7: delay(door-to-*) 계산 함수 정의

def compute_delay_minutes(df, start_col, end_col, new_col, default=-1):
    """
    df[start_col], df[end_col] 이 datetime일 때:
      end - start 를 '분' 단위로 계산.
    - 둘 중 하나라도 결측이면 default
    - 음수(ED 이전에 일어난 event)도 default로 처리
    """
    if start_col not in df.columns or end_col not in df.columns:
        print(f"[WARN] {start_col} 또는 {end_col} 이(가) df에 없어서 {new_col}을 default로 채웁니다.")
        df[new_col] = default
        return df

    diff = (df[end_col] - df[start_col]).dt.total_seconds() / 60.0
    df[new_col] = diff

    # NaN → default
    df[new_col] = df[new_col].fillna(default)

    # 음수 값도 default 처리 (ED 도착 전 event로 간주)
    df.loc[df[new_col] < 0, new_col] = default

    return df


In [119]:
# 셀 8: door_to_ecg / door_to_trop / door_to_anti / door_to_pci 계산

df = compute_delay_minutes(
    df,
    start_col="ed_intime",
    end_col="first_ecg_charttime",
    new_col="door_to_ecg",
    default=-1,
)

df = compute_delay_minutes(
    df,
    start_col="ed_intime",
    end_col="first_troponin_charttime",
    new_col="door_to_trop",
    default=-1,
)

df = compute_delay_minutes(
    df,
    start_col="ed_intime",
    end_col="first_antithrombotic_charttime",
    new_col="door_to_anti",
    default=-1,
)

df = compute_delay_minutes(
    df,
    start_col="ed_intime",
    end_col="pci_starttime",
    new_col="door_to_pci",
    default=-1,
)

df[[
    "subject_id", "hadm_id",
    "door_to_ecg", "door_to_trop",
    "door_to_anti", "door_to_pci"
]].head(10)


,subject_id,hadm_id,door_to_ecg,door_to_trop,door_to_anti,door_to_pci
0,10010058,26359957,-1.0,293.0,250.0,284.0
1,10012438,22764825,-1.0,-1.0,207.0,132.0
2,10013310,21243435,-1.0,349.0,604.0,-1.0
3,10013310,27682188,-1.0,1012.0,519.0,484.0
4,10033552,26487381,-1.0,91.0,591.0,-1.0
5,10055344,29209451,-1.0,328.0,550.0,318.0
6,10067195,21564201,17.0,445.0,450.0,-1.0
7,10078805,20434240,-1.0,595.0,417.0,964.0
8,10096420,25396519,8.0,393.0,-1.0,154.0
9,10130348,24286651,-1.0,681.0,186.0,-1.0


In [120]:
# 셀 9: 최종 12개 컬럼 선택 및 저장

final_cols = [
    "subject_id",
    "hadm_id",
    "age",
    "gender_enc",            # 나중에 gender로 이름 변경
    "race",
    "arrival_transport_enc", # 나중에 arrival_transport로 이름 변경
    "cci_score",
    "hfrs_score",
    "door_to_ecg",
    "door_to_trop",
    "door_to_anti",
    "door_to_pci",
]

for c in final_cols:
    if c not in df.columns:
        raise ValueError(f"[ERROR] 최종 변수 {c} 이(가) df에 없습니다.")

df_out = df[final_cols].copy()

# 컬럼명 리네임
df_out = df_out.rename(columns={
    "gender_enc": "gender",
    "arrival_transport_enc": "arrival_transport",
})

print("[INFO] 최종 df_out shape:", df_out.shape)
display(df_out.head(10))


[INFO] 최종 df_out shape: (1930, 12)


,subject_id,hadm_id,age,gender,race,arrival_transport,cci_score,hfrs_score,door_to_ecg,door_to_trop,door_to_anti,door_to_pci
0,10010058,26359957,80,1,ASIAN - SOUTH EAST ASIAN,2,1,7.1,-1.0,293.0,250.0,284.0
1,10012438,22764825,65,1,WHITE,2,1,3.8,-1.0,-1.0,207.0,132.0
2,10013310,21243435,70,0,BLACK/AFRICAN,0,1,4.5,-1.0,349.0,604.0,-1.0
3,10013310,27682188,70,0,BLACK/AFRICAN,2,4,10.3,-1.0,1012.0,519.0,484.0
4,10033552,26487381,62,0,OTHER,2,1,1.4,-1.0,91.0,591.0,-1.0
5,10055344,29209451,83,1,BLACK/CAPE VERDEAN,0,4,11.6,-1.0,328.0,550.0,318.0
6,10067195,21564201,59,0,WHITE,2,7,2.3,17.0,445.0,450.0,-1.0
7,10078805,20434240,83,1,WHITE,2,4,20.9,-1.0,595.0,417.0,964.0
8,10096420,25396519,65,1,WHITE,2,0,0.0,8.0,393.0,-1.0,154.0
9,10130348,24286651,80,0,BLACK/AFRICAN AMERICAN,1,0,0.0,-1.0,681.0,186.0,-1.0


In [121]:
# ver180 다시 로드
df180 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/cohort_ver180_static_vars.csv")

print("중복 전 shape:", df180.shape)

# hadm_id 기준 완전 중복 제거
df180_dedup = df180.drop_duplicates(subset=["hadm_id"]).copy()

print("중복 제거 후 shape:", df180_dedup.shape)

# 혹시 진짜 잘 지워졌는지 다시 체크
dups = df180_dedup["hadm_id"].value_counts()
print("중복 hadm_id 개수(>1):", (dups > 1).sum())


중복 전 shape: (1930, 12)
중복 제거 후 shape: (1929, 12)
중복 hadm_id 개수(>1): 0


In [123]:
# CSV 저장
df180_dedup.to_csv(OUTPUT_PATH, index=False)
print(f"[INFO] 저장 완료 → {OUTPUT_PATH}")

[INFO] 저장 완료 → /content/drive/MyDrive/Colab Notebooks/cohort_ver180_static_vars.csv


In [124]:
import pandas as pd

# 1) CSV 로딩
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/cohort_ver180_static_vars.csv")

# 2) 분석 대상 delay 컬럼 설정
delay_cols = ["door_to_ecg", "door_to_trop", "door_to_anti", "door_to_pci"]

print("===== 원본 데이터 delay 통계 =====")
print(df[delay_cols].describe(percentiles=[0.5,0.9,0.95,0.99]))
print("\n원본 shape:", df.shape)

# 3) 각 delay 컬럼의 99퍼센타일 계산
cutoffs = {col: df[col].quantile(0.99) for col in delay_cols}

print("\n===== 99th percentile cutoff =====")
for col, val in cutoffs.items():
    print(f"{col}: {val:.3f}")

# 4) 상위 1% 제거한 새로운 DF 생성
filtered = df.copy()
for col in delay_cols:
    cutoff = cutoffs[col]
    filtered = filtered[filtered[col] <= cutoff]

print("\n===== 삭제 후 데이터 shape =====")
print(filtered.shape)

# 5) 삭제 후 delay 분포 출력
print("\n===== 삭제 후 delay 통계 =====")
print(filtered[delay_cols].describe(percentiles=[0.5,0.9,0.95,0.99]))


===== 원본 데이터 delay 통계 =====
        door_to_ecg  door_to_trop  door_to_anti   door_to_pci
count  1.929000e+03   1929.000000   1929.000000   1929.000000
mean   1.125436e+04    560.964749    572.434422    413.646967
std    4.572375e+05   2587.656220    720.428559   1636.466180
min   -1.000000e+00     -1.000000     -1.000000     -1.000000
50%   -1.000000e+00    352.000000    429.000000     -1.000000
90%    3.600000e+01    994.200000   1097.600000    547.400000
95%    1.112000e+02   1279.800000   1608.600000   1818.400000
99%    3.964800e+02   4598.720000   3477.320000   8006.760000
max    2.001780e+07  99390.000000  12805.000000  22383.000000

원본 shape: (1929, 12)

===== 99th percentile cutoff =====
door_to_ecg: 396.480
door_to_trop: 4598.720
door_to_anti: 3477.320
door_to_pci: 8006.760

===== 삭제 후 데이터 shape =====
(1856, 12)

===== 삭제 후 delay 통계 =====
       door_to_ecg  door_to_trop  door_to_anti  door_to_pci
count  1856.000000   1856.000000   1856.000000  1856.000000
mean     14.040948 

In [125]:
# 5) CSV로 저장
output_path = "/content/drive/MyDrive/Colab Notebooks/cohort_ver181_outlier_processing.csv"
filtered.to_csv(output_path, index=False)

print("저장 완료:", output_path)

저장 완료: /content/drive/MyDrive/Colab Notebooks/cohort_ver181_outlier_processing.csv


In [126]:
import pandas as pd

path180 = "/content/drive/MyDrive/Colab Notebooks/cohort_ver180_static_vars.csv"
path181 = "/content/drive/MyDrive/Colab Notebooks/cohort_ver181_outlier_processing.csv"

# 파일 로드
df180 = pd.read_csv(path180)
df181 = pd.read_csv(path181)

# shape 출력
print("===== ROW / COL 비교 =====")
print(f"ver180 → rows: {df180.shape[0]:4d}, cols: {df180.shape[1]}")
print(f"ver181 → rows: {df181.shape[0]:4d}, cols: {df181.shape[1]}")

# 차이도 함께 출력
print("\n===== 차이 (181 - 180) =====")
print(f"Row 변화: {df181.shape[0] - df180.shape[0]}")
print(f"Col 변화: {df181.shape[1] - df180.shape[1]}")


===== ROW / COL 비교 =====
ver180 → rows: 1929, cols: 12
ver181 → rows: 1856, cols: 12

===== 차이 (181 - 180) =====
Row 변화: -73
Col 변화: 0
